In [1]:
# Import relevant packages 

In [ ]:
# Load the data 

In [ ]:
# Visualize the data 

In [ ]:
# Preprocess the data 

In [ ]:
# Create the model? 

In [ ]:
# Train the model 

In [ ]:
# Test the accuracy 

In [ ]:
# Create a csv for Kaggle 